In [169]:
import pandas as pd
import numpy as np

In [170]:
oshima = pd.read_csv('/Users/genie/Documents/COLLABORATION/AirbnbWise/Modeling/Data/Oshima/oshima_data_add_latlng.csv')
display(oshima.head())

,Unnamed: 0,district,address,info,date,full name,latitude,longitude
0,0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309
1,1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166
2,2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623
3,3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401
4,4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044


In [171]:
oshima.drop(columns=['Unnamed: 0'], inplace=True)

In [172]:
#pd.options.display.max_rows = 9999
oshima['info'].value_counts().head(10)

火災による死亡            317
死体発見               238
飛び降り自殺             168
自殺                 142
孤高死                101
告知事項あり（不動産情報より）     94
首吊り自殺               77
転落死                 71
刺殺                  64
告知事項あり              60
Name: info, dtype: int64

In [173]:

#TODO : oshima 범죄 카테고리 상위 10개 
#TODO : < 1차적 목표 > 화재, 시체발견, 자살, 투신자살, 추락사, 부동산 고지사항 카테고리 범주 안에서 태깅 목표(완료)
#TODO : < 2차적 목표 > 상위 10개 제외한 예외 경우 새로운 카테고리 만들어서 태깅
#TODO : < 3차적 목표 > 고지사항의 경우 특정 죽음 키워드(사망, 자살)이 포함된 안내사항 인 경우 태깅하고 그렇지 않으면 데이터 삭제

#? : 告知事項あり（不動産情報より）와 告知事項あり는 같은 부동산 고지사항인데 분류된 기준이 뭐일까? -> 부동산 고지사항에 해당하는 부분 좀 더 파보자.
#* 火災による死亡           317 -> 火災 : 화재
#* 死体発見               238 -> 시체 발견 -> 死体 : 시체 
#* 飛び降り自殺             168 -> 높은 곳에서 뛰어 내려 자살(투신자살) -> 自殺 : 자살, 추가 info
#* 自殺                 142 -> 자살 
#* 孤高死                101 -> 고독사
#* 告知事項あり（不動産情報より）     94 -> 부동산 정보로 나온 정보를 통해 거래 상대방에게 고지한 내용, 좀 더 강조된 걸까?
#* 首吊り自殺               77 -> 목 매달아 자살 -> 자살, 추가 info
#* 転落死                 71 -> 낙하사망(추락사)
#* 刺殺                  64 -> 찔러서 죽인 행위 -> 타살
#* 告知事項あり            60 -> -> 부동산 정보로 나온 정보를 통해 거래 상대방에게 고지한 내용 
#! ['火災', '死体', '自殺', '孤高死', '告知事項あり', '転落死', '刺殺'] -> 화재, 시체, 자살, 고독사, 고지사항, 추락사, 타살

In [174]:
oshima.shape

(4833, 7)

In [175]:
#* 주소 중복 데이터 확인됨
oshima.drop_duplicates(keep='first', subset=['address'], inplace=True)
oshima.shape

(3644, 7)

In [176]:
oshima['info'].value_counts().head(10)

火災による死亡            254
死体発見               178
飛び降り自殺             121
自殺                 106
孤高死                 83
告知事項あり（不動産情報より）     76
首吊り自殺               56
転落死                 51
刺殺                  47
告知事項あり              41
Name: info, dtype: int64

In [177]:
# categoryList = oshima['info'].value_counts().head(10).index.tolist()
# categoryList

In [178]:
#* 상위 10개에 해당하는 데이터와 아닌 데이터로 구분
# oshimaTop10 = oshima[oshima['info'].isin(categoryList)].reset_index(drop=True)
# oshimaNotTop10 = oshima[~oshima['info'].isin(categoryList)].reset_index(drop=True)
# print(oshimaTop10.shape, oshimaNotTop10.shape)

In [179]:
# oshimaTop10.loc[:, 'crimeType'] = np.nan
# oshimaNotTop10.loc[:, 'crimeType'] = np.nan

In [180]:
#* 카테고리 7개로 축소
#! ['火災', '死体', '自殺', '孤高死', '告知事項あり', '転落死', '刺殺'] -> 화재, 시체, 자살, 고독사, 고지사항, 추락사, 타살
# keyword_mapping = {
#     '火災': '화재',
#     '死体': '시체',
#     '自殺': '자살',
#     '孤高死': '고독사',
#     '告知事項あり': '부동산고지',
#     '転落死': '추락사',
#     '刺殺': '타살'
# }

In [181]:
keyword_mapping = {
    
    '火災': '화재',
    '死体': '시체',
    '遺体' : '시체',
    'ミイラ' : '시체',
    '自殺': '자살',
    '首吊り' : '자살',
    '飛び降り' : '자살',
    '無理心中' : '자살',
    '縊死' : '자살',
    '孤高死': '고독사',
    '孤独死' : '고독사',
    'て独' : '고독사',
    '告知事項あり': '부동산고지',
    '心理的瑕疵' : '부동산고지',
    '告知事項有り' : '부동산고지',
    '告知事項有' : '부동산고지',
    '告知事項が御座います' : '부동산고지',
    '告知事項' : '부동산고지',
    '転落死': '추락사',
    'から転落' : '추락사',
    '刺殺': '타살',
    '絞殺' : '타살',
    '射殺' : '타살',
    '車両事故死' : '타살',
    '傷害致死' : '타살',
    '撲殺' : '타살',
    '殺人' : '타살',
    '殴殺' : '타살',
    '虐待' : '타살',
    '病死' : '질병사',
    '過労死' : '질병사',
    '熱中症による死亡' : '질병사',
    '事故死' : '사고사',
    'エレベータと壁に挟まれ死亡' : '사고사',
    '変死' : '의문사',
    '不審死' : '의문사', 
    '原因不明の事故' : '의문사',
    '精神的瑕疵' : '심신미약'
    
}




In [182]:
oshima.loc[:, 'crimeType'] = np.nan

for idx in range(oshima.shape[0]):
    for key, value in keyword_mapping.items():
        try:
            if key in str(oshima.loc[idx, 'info']):
                oshima.loc[idx, 'crimeType'] = value
                break
        except KeyError:
                continue
oshima

,district,address,info,date,full name,latitude,longitude,crimeType
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,시체
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,자살
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,시체
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,자살
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,자살
...,...,...,...,...,...,...,...,...
4816,杉並区,東京都杉並区善福寺三丁目6-13,心理的瑕疵あり,不明,東京都 杉並区 善福寺 三丁目 6番 13号,35.716324,139.592682,NaN
4817,杉並区,東京都杉並区上井草三丁目37-15プチコ－トＵ201号室,「心理的瑕疵あり」と説明あり (LIFULL HOME'S - エイブル上石神井店),不明,東京都 杉並区 上井草 三丁目 37番 15号,35.723637,139.600052,NaN
4818,多摩市,東京都多摩市諏訪四丁目3-3都営諏訪団地4-3-3号棟105,自殺&死体発見,平成24年8月7日,東京都 多摩市 諏訪 四丁目 3番,35.622181,139.458511,NaN
4819,多摩市,東京都多摩市永山四丁目3-14,多摩ニュータウン永山団地4-3-14号棟102号室\r\n特別募集住宅,不明,東京都 多摩市 永山 四丁目 3番,35.617443,139.450256,NaN


In [185]:
oshima.loc[(oshima['info'] == '火災による死亡')].tail(50) #! mapping이 왜 안될까?

,district,address,info,date,full name,latitude,longitude,crimeType
3984,東村山市,埼玉県所沢市下安松379-1森田コーポ305,火災による死亡,平成29年6月30日,埼玉県 所沢市 大字下安松 379番地,35.782562,139.498962,NaN
4026,国立市,東京都国分寺市内藤一丁目5-27,火災による死亡,令和3年4月13日,東京都 国分寺市 内藤 一丁目 5番,35.693951,139.457794,NaN
4090,板橋区,東京都板橋区新河岸三丁目8-14,火災による死亡,平成26年3月15日,東京都 板橋区 新河岸 三丁目 8番 14号,35.796040,139.648804,NaN
4092,板橋区,東京都板橋区新河岸三丁目8-18,火災による死亡,平成24年2月14日,東京都 板橋区 新河岸 三丁目 8番 18号,35.796234,139.648178,NaN
4168,板橋区,東京都板橋区若木二丁目9-7千種荘202,火災による死亡,平成24年12月25日,東京都 板橋区 若木 二丁目 9番,35.770882,139.674606,NaN
4174,板橋区,東京都板橋区成増四丁目20-6 N・Eハイム1階,火災による死亡,平成29年11月4日,東京都 板橋区 成増 四丁目 20番 6号,35.784996,139.633835,NaN
4184,世田谷区,東京都杉並区上高井戸一丁目32-57,火災による死亡,平成18年12月21日,東京都 杉並区 上高井戸 一丁目 32番 57号,35.676052,139.609451,NaN
4192,世田谷区,東京都世田谷区南烏山三丁目14-6,火災による死亡,平成18年1月13日,東京都 世田谷区 南烏山 三丁目 14番 6号,35.672611,139.605148,NaN
4199,世田谷区,東京都世田谷区八幡山三丁目37-2都営八幡山アパート2号棟404,火災による死亡,平成26年12月9日,東京都 世田谷区 八幡山 三丁目 37番 2号,35.663399,139.614334,NaN
4209,世田谷区,東京都世田谷区上北沢四丁目16-12小宮マンション301,火災による死亡,平成22年5月23日,東京都 世田谷区 上北沢 四丁目 16番 12号,35.669937,139.624802,NaN
